In [2]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb


In [9]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
wrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)


In [23]:
editor = checklist.editor.Editor()
editor.tg

In [26]:
editor.suggest('I really {mask} the flight.')

['enjoyed',
 'liked',
 'loved',
 'appreciated',
 'missed',
 'enjoy',
 'needed',
 'appreciate',
 'miss',
 'regret',
 'took',
 'like',
 'regretted',
 'survived',
 'hated',
 'love',
 'enjoying',
 'wanted',
 'got',
 'worth',
 'recommend',
 'preferred',
 'made',
 'need',
 'disliked',
 'hate',
 'had',
 'planned',
 'felt',
 'remember',
 'used',
 'finished',
 'drove',
 'tried',
 'did',
 'cherished',
 'rocked',
 'saved',
 'deserved',
 'crashed',
 'managed',
 'left',
 'earned',
 'helped',
 'ended',
 'rode',
 'underestimated',
 'was',
 'prefer',
 'regrets',
 'bought',
 'cherish',
 'enjoys',
 'spoiled',
 'packed',
 'spent',
 'followed',
 'ruined',
 'valued',
 'timed',
 'delayed',
 'recommended',
 'handled',
 'improved',
 'take',
 'caught',
 'dreaded',
 'watched',
 'anticipated',
 'supported',
 'likes',
 'admired',
 'canceled',
 'rushed',
 'changed',
 'shortened',
 'reviewed',
 'edited',
 'doubt',
 'value',
 'blessed',
 'understood',
 'feel',
 'hit',
 'beat',
 'respected',
 'for',
 'worked',
 'flew

In [25]:
editor.tg.unmask('I really <mask> the flight.')

[(['enjoyed'], ' I really enjoyed the flight.', 21.853195190429688),
 (['liked'], ' I really liked the flight.', 18.678905487060547),
 (['loved'], ' I really loved the flight.', 17.917505264282227),
 (['appreciated'], ' I really appreciated the flight.', 17.617549896240234),
 (['missed'], ' I really missed the flight.', 17.388092041015625),
 (['enjoy'], ' I really enjoy the flight.', 16.8438720703125),
 (['needed'], ' I really needed the flight.', 16.730182647705078),
 (['appreciate'], ' I really appreciate the flight.', 16.136600494384766),
 (['miss'], ' I really miss the flight.', 15.77472972869873),
 (['regret'], ' I really regret the flight.', 15.708377838134766),
 (['took'], ' I really took the flight.', 15.481184959411621),
 (['like'], ' I really like the flight.', 15.384997367858887),
 (['regretted'], ' I really regretted the flight.', 15.06374454498291),
 (['survived'], ' I really survived the flight.', 14.90787124633789),
 (['hated'], ' I really hated the flight.', 14.86866855

In [4]:
# editor.template((('{first_name} is a good guy', 'Who is a good guy?'), '{first_name}', '3') )

In [5]:
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
sentences = tdata
parsed_data = list(nlp.pipe(sentences))

In [10]:
def new_pp(data):
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    pr = wrapped_pp(data)[1][:, 1]
    pp = np.zeros((pr.shape[0], 3))
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

In [9]:
suite = TestSuite()

## Aspect: Vocabulary

### MFTs

In [10]:
air_noun = ['flight', 'seat', 'pilot', 'staff', 'service', 'customer service', 'aircraft', 'plane', 'food', 'cabin crew', 'company', 'airline', 'crew']
editor.add_lexicon('air_noun', air_noun)

In [11]:
print(', '.join(editor.suggest('It was {a:mask} {air_noun}.')[:40]))

great, good, excellent, amazing, bad, terrible, incredible, awful, expensive, extraordinary, new, interesting, wonderful, nice, fantastic, real, important, awesome, unusual, American, different, poor, exceptional, beautiful, ordinary, little, impressive, special, enormous, actual, easy, big, horrible, huge, fine, lousy, perfect, international, decent, terrific


In [12]:
pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'weird', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']
neutral_adj = ['American', 'international',  'commercial', 'British', 'private', 'Italian', 'Indian', 'Australian', 'Israeli', ]
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)

In [13]:
print(', '.join(editor.suggest('I really {mask} the {air_noun}.')[:200]))
# print()
# print(', '.join(editor.suggest('I {mask} the {air_noun}.')[:50]))

liked, enjoyed, like, appreciate, appreciated, enjoy, loved, love, miss, missed, hate, needed, likes, wanted, got, recommend, prefer, admired, value, need, respected, want, dislike, enjoying, respect, enjoys, admire, was, feel, liking, dig, hated, mean, preferred, underestimated, disliked, used, get, use, valued, understand, did, found, adore, helped, dug, trust, remember, noticed, about, tried, hit, had, regret, took, felt, praised, cherish, have, left, loves, understood, do, LOVE, compliment, trusted, bought, thank, treasure, applaud, support, rate, know, commend, credit, underestimate, see, supported, think, thought, all, saw, picked, believe, beat, welcome, considered, impressed, improved, met, chose, thanks, deserved, envy, blame, for, help, packed, recommended, in, follow, choose, loving, misses, is, into, lost, consider, changed, worked, meant, leave, joined, wish, regretted, made, earned, rocked, cherished, take, worth, finished, experienced, are, fancy, handled, thanked, follo

In [14]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise' ]
neutral_verb_present = ['see', 'find']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']
neutral_verb_past = ['saw', 'found']
editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

Individual words

In [15]:
test = MFT(pos_adj + pos_verb_present + pos_verb_past, labels=2)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'single positive words', 'Vocabulary', '')

In [16]:
test = MFT(neg_adj + neg_verb_present + neg_verb_past, labels=0)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'single negative words', 'Vocabulary', '')

In [17]:
test = MFT(neutral_adj + neutral_verb_present + neutral_verb_past, labels=1)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'single neutral words', 'Vocabulary', 'TODO_DESCRIPTION')

Words in context

In [18]:
t = editor.template('{it} {air_noun} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'], labels=2, save=True)
t += editor.template('{it} {be} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=2, save=True)
t += editor.template('{i} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=2, save=True)
t += editor.template('{it} {air_noun} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{it} {be} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=0, save=True)
t += editor.template('{i} {neg_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=0, save=True)
# equivalent to:
# test = MFT(t.data, labels=t.labels, templates=t.templates)
test = MFT(**t)
suite.add(test, 'Sentiment-laden words in context', 'Vocabulary', 'Use positive and negative verbs and adjectives with airline nouns such as seats, pilot, flight, etc. E.g. "This was a bad flight"')


In [19]:
editor.lexicons['neutral_verb']

['see', 'find', 'saw', 'found']

In [20]:
t = editor.template('{it} {air_noun} {be} {neutral_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], save=True)
t += editor.template('{it} {be} {a:neutral_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], save=True)
t += editor.template('{i} {neutral_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'neutral words in context', 'Vocabulary', 'Use neutral verbs and adjectives with airline nouns such as seats, pilot, flight, etc. E.g. "The pilot is American"')

### Intensifiers and reducers

In [21]:
print(' , '.join(editor.suggest('{it} {be} {a:mask} {pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])[:50]))

absolutely , really , very , truly , pretty , just , simply , quite , most , incredibly , totally , amazingly , extremely , absolute , almost , rather , completely , super , utterly , especially , unbelievably , equally , surprisingly , exceptionally , extraordinarily , particularly , overall , entirely , genuinely , otherwise , insanely , obviously , fucking , fairly , altogether , amazing , actually , undeniably , all , unusually , damn , incredible , exceedingly , awesome , already , entire , an , oddly , extra , actual


In [22]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'amazingly', 'especially', 'exceptionally', 'unbelievably', 'utterly', 'exceedingly', 'rather', 'totally', 'particularly']

In [23]:
print(', '.join(editor.suggest('{i} {mask} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])[:100]))

really, always, also, greatly, truly, certainly, I, thoroughly, especially, still, personally, particularly, highly, all, definitely, we, sincerely, very, much, most, so, deeply, fully, both, absolutely, just, and, people, strongly, simply, obviously, do, actually, have, clearly, quite, sure, rather, too, you, generally, they, would, did, genuinely, will, again, completely, totally, honestly, never, seriously, should, even, guys, only, often, can, must, to, now, who, immediately, many, extremely, then, does, mostly, immensely, friends, dearly, everyone, kindly, usually, shall, ever, may, respectfully, tremendously, already, hugely, students, probably, had, please, least, desperately, incredibly, that, long, REALLY, enthusiastically, could, he, secretly, therefore, readers, me, family, further


In [33]:
editor.visual_suggest('I really {mask} the flight.')

TemplateEditor(bert_suggests=['enjoyed', 'liked', 'loved', 'appreciated', 'missed', 'enjoy', 'needed', 'apprec…

In [24]:
intens_verb = [ 'really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 'highly', 'definitely', 'certainly', 'genuinely', 'honestly', 'strongly', 'sure', 'sincerely']

In [25]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)

In [26]:
t = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
t.data[:5]


[['That was an incredible plane.',
  'That was an exceptionally incredible plane.'],
 ['That was an exciting plane.', 'That was an amazingly exciting plane.'],
 ['It was a beautiful service.', 'It was a particularly beautiful service.'],
 ['It was an exceptional seat.', 'It was a rather exceptional seat.'],
 ['It was a happy customer service.',
  'It was an incredibly happy customer service.']]

In [27]:
t = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
test = DIR(t.data, monotonic_label, templates=t.templates)
description = '''Test is composed of pairs of sentences (x1, x2), where we add an intensifier
such as "really",or "very" to x2 and expect the confidence to NOT go down (with tolerance=0.1). e.g.:
x1 = "That was a good flight"
x2 = "That was a very good flight"
We disregard cases where the prediction of x1 is neutral.
'''
suite.add(test, 'intensifiers', 'Vocabulary', description)
# test.run(new_pp)
# test.summary(3)


In [28]:
reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'reasonably', 'a little', 'a bit', 'slightly']

In [29]:
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_neutral_pred)

In [30]:

t = editor.template(['{it} {air_noun} {be} {pos_adj}.', '{it} {air_noun} {be} {red} {pos_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000, save=True)
t += editor.template(['{it} {air_noun} {be} {neg_adj}.', '{it} {air_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000, save=True)
t.data[:50]

[['This customer service was sweet.',
  'This customer service was kinda sweet.'],
 ['The customer service is nice.', 'The customer service is reasonably nice.'],
 ['This seat is adorable.', 'This seat is generally adorable.'],
 ['That customer service was fantastic.',
  'That customer service was kinda fantastic.'],
 ['That food was wonderful.', 'That food was slightly wonderful.'],
 ['That aircraft was beautiful.', 'That aircraft was generally beautiful.'],
 ['That flight was sweet.', 'That flight was generally sweet.'],
 ['This aircraft is wonderful.', 'This aircraft is a little wonderful.'],
 ['The crew was perfect.', 'The crew was slightly perfect.'],
 ['The customer service is incredible.',
  'The customer service is probably incredible.'],
 ['This customer service is nice.',
  'This customer service is generally nice.'],
 ['This customer service is brilliant.',
  'This customer service is kinda brilliant.'],
 ['That service is great.', 'That service is slightly great.'],
 ['The 

In [31]:
t = editor.template(['{it} {air_noun} {be} {pos_adj}.', '{it} {air_noun} {be} {red} {pos_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000, save=True)
t += editor.template(['{it} {air_noun} {be} {neg_adj}.', '{it} {air_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000, save=True)
test = DIR(t.data, monotonic_label_down, templates=t.templates)
description = '''Test is composed of pairs of sentences (x1, x2), where we add a reducer
such as "somewhat", or "kinda" to x2 and expect the confidence to NOT go up (with tolerance=0.1). e.g.:
x1 = "The cabin crew was good."
x2 = "The cabin crew was somewhat good."
We disregard cases where the prediction of x1 is neutral.
'''
suite.add(test, 'reducers', 'Vocabulary', description)
# test.run(new_pp)
# test.summary(3)


### INVariance: change neutral words

In [32]:
neutral_words = set(
    ['.', 'the', 'The', ',', 'a', 'A', 'and', 'of', 'to', 'it', 'that', 'in',
     'this', 'for',  'you', 'there', 'or', 'an', 'by', 'about', 'flight', 'my',
     'in', 'of', 'have', 'with', 'was', 'at', 'it', 'get', 'from', 'this', 'Flight', 'plane'
    ])
forbidden = set(['No', 'no', 'Not', 'not', 'Nothing', 'nothing', 'without', 'but'] + pos_adj + neg_adj + pos_verb_present + pos_verb_past + neg_verb_present + neg_verb_past)
def change_neutral(d):
#     return d.text
    examples = []
    subs = []
    words_in = [x for x in d.capitalize().split() if x in neutral_words]
    if not words_in:
        return None
    for w in words_in:
        suggestions = [x for x in editor.suggest_replace(d, w, beam_size=5, words_and_sentences=True) if x[0] not in forbidden]
        examples.extend([x[1] for x in suggestions])
        subs.extend(['%s -> %s' % (w, x[0]) for x in suggestions])
    if examples:
        idxs = np.random.choice(len(examples), min(len(examples), 10), replace=False)
        return [examples[i] for i in idxs]#, [subs[i] for i in idxs])
# Perturb.perturb(parsed_data[:5], perturb)

In [33]:
t = Perturb.perturb(sentences, change_neutral, nsamples=500)
test = INV(t.data)
description = 'Change a set of neutral words with other context-appropriate neutral words (using BERT).'
suite.add(test, 'change neutral words with BERT', 'Vocabulary', description)
# test.run(new_pp)
# test.summary(3)

### Add negative phrases

In [34]:
positive = editor.template('I {pos_verb_present} you.').data
positive += editor.template('You are {pos_adj}.').data
positive += ['I would fly with you again.']
positive.remove('You are happy.')
negative = editor.template('I {neg_verb_present} you.').data
negative += editor.template('You are {neg_adj}.').data
negative += ['Never flying with you again.']
def add_phrase_function(phrases):
    def pert(d):
        while d[-1].pos_ == 'PUNCT':
            d = d[:-1]
        d = d.text
        ret = [d + '. ' + x for x in phrases]
        idx = np.random.choice(len(ret), 10, replace=False)
        ret = [ret[i] for i in idx]
        return ret
    return pert


In [86]:
def positive_change(orig_conf, conf):
    softmax = type(orig_conf) in [np.array, np.ndarray]
    if not softmax or orig_conf.shape[0] != 3:
        raise(Exception('Need prediction function to be softmax with 3 labels (negative, neutral, positive)'))
    return orig_conf[0] - conf[0] + conf[2] - orig_conf[2]

def diff_up(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    tolerance = 0.1
    change = positive_change(orig_conf, conf)
    if change + tolerance >= 0:
        return True
    else:
        return change + tolerance
def diff_down(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    tolerance = 0.1
    change = positive_change(orig_conf, conf)
    if change - tolerance <= 0:
        return True
    else:
        return -(change - tolerance)
goes_up = Expect.pairwise(diff_up)
goes_down = Expect.pairwise(diff_down)
    

In [36]:
t = Perturb.perturb(parsed_data, add_phrase_function(positive), nsamples=500)
test = DIR(t.data, goes_up)
description = 'Add very positive phrases (e.g. I love you) to the end of sentences, expect probability of positive to NOT go down (tolerance=0.1)'
suite.add(test, 'add positive phrases', 'Vocabulary', description)
# test.run(new_pp, overwrite=True)
# test.summary(3)


In [37]:
t = Perturb.perturb(parsed_data, add_phrase_function(negative), nsamples=500)
test = DIR(t.data, goes_down)
description = 'Add very negative phrases (e.g. I hate you) to the end of sentences, expect probability of positive to NOT go up (tolerance=0.1)'
suite.add(test, 'add negative phrases', 'Vocabulary', description)
# test.run(new_pp, overwrite=True)
# test.summary(3)


## Aspect: robustness
### INVariance: adding irrelevant stuff before and after.


In [38]:
import string
def random_string(n):
    return ''.join(np.random.choice([x for x in string.ascii_letters + string.digits], n))
def random_url(n=6):
    return 'https://t.co/%s' % random_string(n)
def random_handle(n=6):
    return '@%s' % random_string(n)

# data['sentence']

def add_irrelevant(sentence):
    urls_and_handles = [random_url(n=6) for _ in range(5)] + [random_handle() for _ in range(5)]
    irrelevant_before = ['@airline '] + urls_and_handles
    irrelevant_after = urls_and_handles 
    rets = ['%s %s' % (x, sentence) for x in irrelevant_before ]
    rets += ['%s %s' % (sentence, x) for x in irrelevant_after]
    return rets


In [39]:
t = Perturb.perturb(sentences, add_irrelevant, nsamples=500)
test = INV(t.data)
suite.add(test, 'add random urls and handles', 'Robustness', 'add randomly generated urls and handles to the start or end of sentence')
# test.run(new_pp)
# test.summary(3)

### punctuation, contractions, typos

In [40]:
t = Perturb.perturb(parsed_data, Perturb.punctuation, nsamples=500)
test = INV(t.data)
suite.add(test, 'punctuation', 'Robustness', 'strip punctuation and / or add "."')
# test.run(new_pp)
# test.summary(3)

In [41]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=1)
test = INV(t.data)
suite.add(test, 'typos', 'Robustness', 'Add one typo to input by swapping two adjacent characters')
# test.run(new_pp)
# test.summary(3)

In [42]:
t = Perturb.perturb(sentences, Perturb.add_typos, nsamples=500, typos=2)
test = INV(t.data)
suite.add(test, '2 typos', 'Robustness', 'Add two typos to input by swapping two adjacent characters twice')
# test.run(new_pp)
# test.summary(3)


In [43]:
t = Perturb.perturb(sentences, Perturb.contractions, nsamples=1000)
test = INV(t.data)
suite.add(test, 'contractions', 'Robustness', 'Contract or expand contractions, e.g. What is -> What\'s')
# test.run(new_pp)
# test.summary(3)

## Aspect: NER

In [44]:
t = Perturb.perturb(parsed_data, Perturb.change_names, nsamples=1000)
test = INV(t.data)
suite.add(test, 'change names', 'NER', 'Replace names with other common names')
# test.run(new_pp)
# test.summary(3)

In [45]:
t = Perturb.perturb(parsed_data, Perturb.change_location, nsamples=1000)
test = INV(t.data)
suite.add(test, 'change locations', 'NER', 'Replace city or country names with other cities or countries')
# test.run(new_pp)
# test.summary(3)

In [46]:
t = Perturb.perturb(parsed_data, Perturb.change_number, nsamples=1000)
test = INV(t.data)
suite.add(test, 'change numbers', 'NER', 'Replace integers with random integers within a 20% radius of the original')
# test.run(new_pp)
# test.summary(3)

## Aspect: temporal awareness

In [47]:
editor.template('{neg_verb_present}').data

['hate', 'dislike', 'regret', 'abhor', 'dread', 'despise']

In [48]:
change = ['but', 'even though', 'although', '']
t = editor.template(['I used to think this airline was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} I used to think it was {neg_adj}.',
                                 'In the past I thought this airline was {neg_adj}, {change} now I think it is {pos_adj}.',
                                 'I think this airline is {pos_adj}, {change} in the past I thought it was {neg_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=2)
t += editor.template(['I used to {neg_verb_present} this airline, {change} now I {pos_verb_present} it.',
                                 'I {pos_verb_present} this airline, {change} I used to {neg_verb_present} it.',
                                 'In the past I would {neg_verb_present} this airline, {change} now I {pos_verb} it.',
                                 'I {pos_verb_present} this airline, {change} in the past I would {neg_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500, save=True, labels=2)

t += editor.template(['I used to think this airline was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this airline is {neg_adj}, {change} I used to think it was {pos_adj}.',
                                 'In the past I thought this airline was {pos_adj}, {change} now I think it is {neg_adj}.',
                                 'I think this airline is {neg_adj}, {change} in the past I thought it was {pos_adj}.',
                                ] ,
                                 change=change, unroll=True, nsamples=500, save=True, labels=0)
t += editor.template(['I used to {pos_verb_present} this airline, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this airline, {change} I used to {pos_verb_present} it.',
                                 'In the past I would {pos_verb_present} this airline, {change} now I {neg_verb_present} it.',
                                 'I {neg_verb_present} this airline, {change} in the past I would {pos_verb_present} it.',
                                ] ,
                                change=change, unroll=True, nsamples=500, save=True, labels=0)
test = MFT(**t)
description = '''Have two conflicing statements, one about the past and one about the present.
Expect the present to carry the sentiment. Examples:
I used to love this airline, now I hate it -> should be negative
I love this airline, although I used to hate it -> should be positive
'''
suite.add(test, 'used to, but now', 'Temporal', description)
# test.run(new_pp)
# test.summary(n=3)



used to should reduce

In [49]:
t = editor.template(['{it} {be} {a:adj} {air_noun}.', 'I used to think {it} {be} {a:adj} {air_noun}.'], it=['it', 'this', 'that'], be=['is', 'was'], adj=editor.lexicons['pos_adj'] + editor.lexicons['neg_adj'], save=True)
t += editor.template(['{i} {verb} {the} {air_noun}.', '{i} used to {verb} {the} {air_noun}.'], i=['I', 'We'], the=['this', 'that', 'the'], verb=editor.lexicons['pos_verb_present'] + editor.lexicons['neg_verb_present'], save=True)
t.data[:5]


[['it is a good flight.', 'I used to think it is a good flight.'],
 ['this is a good flight.', 'I used to think this is a good flight.'],
 ['that is a good flight.', 'I used to think that is a good flight.'],
 ['it was a good flight.', 'I used to think it was a good flight.'],
 ['this was a good flight.', 'I used to think this was a good flight.']]

In [50]:
t = editor.template(['{it} {be} {a:adj} {air_noun}.', 'I used to think {it} {be} {a:adj} {air_noun}.'], it=['it', 'this', 'that'], be=['is', 'was'], adj=editor.lexicons['pos_adj'] + editor.lexicons['neg_adj'], save=True)
t += editor.template(['{i} {verb} {the} {air_noun}.', '{i} used to {verb} {the} {air_noun}.'], i=['I', 'We'], the=['this', 'that', 'the'], verb=editor.lexicons['pos_verb_present'] + editor.lexicons['neg_verb_present'], save=True)
test = DIR(t.data, monotonic_label_down, templates=t.templates)
# test = MFT(data, labels=2)
suite.add(test, '"used to" should reduce', 'Temporal', 'A model should not be more confident on "I used to think X" when compared to "X", e.g. "I used to love this airline" should have less confidence than "I love this airline"')
# test.run(new_pp)
# test.summary(n=3)

### Aspect:fairness

In [51]:
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'air_noun', 'pos_adj', 'neg_adj', 'neutral_adj', 'pos_verb_present', 'neg_verb_present', 'neutral_verb_present', 'pos_verb_past', 'neg_verb_past', 'neutral_verb_past', 'pos_verb', 'neg_verb', 'neutral_verb'])

In [52]:
editor.template('{a:religion_adj}').data

['a Christian',
 'a Protestant',
 'a Roman Catholic',
 'an Eastern Orthodox',
 'an Anglican',
 'a Jew',
 'an Orthodox Jew',
 'a Muslim',
 'a Sunni',
 "a Shi'a",
 'an Ahmadiyya',
 'a Buddhist',
 'a Zoroastrian',
 'a Hindu',
 'a Sikh',
 'a Shinto',
 "a Baha'i",
 'a Taoist',
 'a Confucian',
 'a Jain',
 'an Atheist',
 'an Agnostic']

In [53]:
protected = {
    'race': ['a black','a hispanic', 'a white', 'an asian'],
    'sexual': editor.template('{a:sexual_adj}').data,
    'religion': editor.template('{a:religion_adj}').data,
    'nationality': editor.template('{a:nationality}').data[:20],
}

In [54]:
# t.data[:10]

In [55]:
for p, vals in protected.items():
    print(p)
    t = editor.template(['{male} is %s {mask}.' % r for r in vals], return_maps=False, nsamples=300, save=True)
    t += editor.template(['{female} is %s {mask}.' % r for r in vals], return_maps=False, nsamples=300, save=True)
    test = INV(t.data, threshold=0.1, templates=t.templates)
    suite.add(test, 'protected: %s' % p, 'Fairness', 'Prediction should be the same for various adjectives within a protected class')
#     test.run(new_pp)
#     test.summary(n=3)
#     print()
#     preds = np.array(test.results.preds)
#     for i, x in enumerate(vals):
#         print('%.2f %s' % (preds[:, i].mean(), vals[i]))
#     print()
#     print()
#     print('-------------------------')

race
sexual
religion
nationality



### Aspect: Negation

Simple templates:

In [73]:
t = editor.template('{it} {air_noun} {nt} {pos_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {pos_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
t += editor.template('No one {pos_verb_present}s {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'simple negations: negative', 'Negation', 'Very simple negations of positive statements')
# test.run(new_pp)
# test.summary(n=3)

In [57]:
t = editor.template('{it} {air_noun} {nt} {neg_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neg_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
t += editor.template('No one {neg_verb_present}s {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
is_not_0 = lambda x, pred, *args: pred != 0
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
suite.add(test, 'simple negations: not negative', 'Negation', 'Very simple negations of negative statements. Expectation requires prediction to NOT be negative (i.e. neutral or positive)')
# test.run(new_pp)
# test.summary(n=3)


In [58]:
t = editor.template('{it} {air_noun} {nt} {neutral_adj}.', it=['This', 'That', 'The'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('{it} {benot} {a:neutral_adj} {air_noun}.', it=['It', 'This', 'That'], benot=['is not',  'isn\'t', 'was not', 'wasn\'t'], save=True)
neg = ['I can\'t say I', 'I don\'t', 'I would never say I', 'I don\'t think I', 'I didn\'t' ]
t += editor.template('{neg} {neutral_verb_present} {the} {air_noun}.', neg=neg, the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'simple negations: not neutral is still neutral', 'Negation', 'Negating neutral statements should still result in neutral predictions')
# test.run(new_pp)
# test.summary(n=3)

Different templates:

In [59]:
air_noun_it = [x for x in editor.lexicons['air_noun'] if x != 'pilot']
t = editor.template('I thought {it} {air_noun} would be {pos_adj}, but it {neg}.', air_noun=air_noun_it, neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('I thought I would {pos_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'simple negations: I thought x was positive, but it was not (should be negative)', 'Negation', '', overwrite=True)
# test.run(new_pp)
# test.summary(n=3)

In [60]:
t = editor.template('I thought {it} {air_noun} would be {neg_adj}, but it {neg}.', air_noun=air_noun_it, neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('I thought I would {neg_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
suite.add(test, 'simple negations: I thought x was negative, but it was not (should be neutral or positive)', 'Negation', '')
# test.run(new_pp)
# test.summary(n=3)


In [61]:
t = editor.template('I thought {it} {air_noun} would be {neutral_adj}, but it {neg}.', air_noun=air_noun_it, neg=['was not', 'wasn\'t'], it=['this', 'that', 'the'], nt=['is not', 'isn\'t'], save=True)
t += editor.template('I thought I would {neutral_verb_present} {the} {air_noun}, but I {neg}.', neg=['did not', 'didn\'t'], the=['this', 'that', 'the'], save=True)
# expectation: prediction is not 0
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'simple negations: but it was not (neutral) should still be neutral', 'Negation', '')
# test.run(new_pp)
# test.summary(n=3)


Harder: negation with neutral in the middle

In [62]:
new_neg = neg[:-1]
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
t = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {pos_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:pos_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {pos_verb_present} {the} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we'], the=['this', 'that', 'the'], save=True)
t.data = list(np.random.choice(t.data, 1000, replace=False))
test = MFT(t.data, labels=0, templates=t.templates)
suite.add(test, 'Hard: Negation of positive with neutral stuff in the middle (should be negative)', 'Negation', '')
# test.run(new_pp)
# test.summary(n=3)

In [63]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
t = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {neg_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neg_adj} {air_noun}.',neutral=neutral,  neg=['i don\'t think', 'i can\'t say', 'i wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neg_verb_present} {the} {air_noun}.',neutral=neutral,  neg=['i don\'t think', 'i can\'t say', 'i wouldn\'t say'], i=['I', 'we'], the=['this', 'that', 'the'], save=True)
t.data = list(np.random.choice(t.data, 1000, replace=False))
test = MFT(t.data, Expect.single(is_not_0), templates=t.templates)
suite.add(test, 'Hard: Negation of negative with neutral stuff in the middle (should be positive or neutral)', 'Negation', '')
# test.run(new_pp)
# test.summary(n=3)


In [64]:
neutral =['that I am from Brazil', 'my history with airplanes', 'all that I\'ve seen over the years', 'the time that I\'ve been flying', 'it\'s a Tuesday']
t = editor.template('{neg}, given {neutral}, that {it} {air_noun} {be} {neutral_adj}.', neutral=neutral, neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {it} {be} {a:neutral_adj} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], it=['this', 'that', 'the'], be=['is', 'was'], save=True)
t += editor.template('{neg}, given {neutral}, that {i} {neutral_verb_present} {the} {air_noun}.',neutral=neutral,  neg=['I don\'t think', 'I can\'t say', 'I wouldn\'t say'], i=['I', 'we'], the=['this', 'that', 'the'], save=True)
t.data = list(np.random.choice(t.data, 1000, replace=False))
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'negation of neutral with neutral in the middle, should still neutral', 'Negation', '')
# test.run(new_pp)
# test.summary(n=3)


### Aspect: SRL

my opinion is more important than others

In [65]:
change = [' but', '']
templates = ['Some people think you are {neg_adj},{change} I think you are {pos_adj}.',
             'I think you are {pos_adj},{change} some people think you are {neg_adj}.',
             'I had heard you were {neg_adj},{change} I think you are {pos_adj}.',
             'I think you are {pos_adj},{change} I had heard you were {neg_adj}.',
             ]
t = editor.template(templates, change=change, unroll=True, labels=2, save=True)
templates = ['{others} {neg_verb_present} you,{change} I {pos_verb_present} you.',
             'I {pos_verb_present} you,{change} {others} {neg_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
t += editor.template(templates, others=others, change=change, unroll=True, labels=2, save=True)

change = [' but', '']
templates = ['Some people think you are {pos_adj},{change} I think you are {neg_adj}.',
             'I think you are {neg_adj},{change} some people think you are {pos_adj}.',
             'I had heard you were {pos_adj},{change} I think you are {neg_adj}.',
             'I think you are {neg_adj},{change} I had heard you were {pos_adj}.',
             ]
t += editor.template(templates, change=change, unroll=True, labels=0, save=True)
templates = ['{others} {pos_verb_present} you,{change} I {neg_verb_present} you.',
             'I {neg_verb_present} you,{change} {others} {pos_verb_present} you.',
            ]
others = ['some people', 'my parents', 'my friends', 'people']
t += editor.template(templates, others=others, change=change, unroll=True, labels=0, save=True)
test = MFT(**t)
description = '''Have conflicting statements where the author has an opinion and a third party has a contrary opinion.
Expect sentiment to be the authors'. Example:
"Some people think you are great, but I think you are terrible" -> should be negative
'''
suite.add(test, 'my opinion is what matters', 'SRL', description)
# test.run(new_pp)
# test.summary(n=3)

q & a form: yes

In [66]:
t = editor.template('Do I think {it} {air_noun} {be} {pos_adj}? Yes', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=2)
t += editor.template('Do I think {it} {be} {a:pos_adj} {air_noun}? Yes', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=2)
t += editor.template('Did {i} {pos_verb_present} {the} {air_noun}? Yes', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=2)
t += editor.template('Do I think {it} {air_noun} {be} {neg_adj}? Yes', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Do I think {it} {be} {a:neg_adj} {air_noun}? Yes', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Did {i} {neg_verb_present} {the} {air_noun}? Yes', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=0)
test = MFT(**t)
suite.add(test, 'Q & A: yes', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

In [67]:
t = editor.template('Do I think {it} {air_noun} {be} {neutral_adj}? Yes', it=['that', 'this', 'the'], be=['is', 'was'], save=True)
t += editor.template('Do I think {it} {be} {a:neutral_adj} {air_noun}? Yes', it=['it', 'this', 'that'], be=['is', 'was'], save=True)
t += editor.template('Did {i} {neutral_verb_present} {the} {air_noun}? Yes', i=['I', 'we'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'Q & A: yes (neutral)', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)


In [68]:
t = editor.template('Do I think {it} {air_noun} {be} {pos_adj}? No', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Do I think {it} {be} {a:pos_adj} {air_noun}? No', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=0)
t += editor.template('Did {i} {pos_verb_present} {the} {air_noun}? No', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=0)
t += editor.template('Do I think {it} {air_noun} {be} {neg_adj}? No', it=['that', 'this', 'the'], be=['is', 'was'], save=True, labels=1)
t += editor.template('Do I think {it} {be} {a:neg_adj} {air_noun}? No', it=['it', 'this', 'that'], be=['is', 'was'], save=True, labels=1)
t += editor.template('Did {i} {neg_verb_present} {the} {air_noun}? No', i=['I', 'we'], the=['this', 'that', 'the'], save=True, labels=1)
allow_for_neutral = lambda x, pred, _, label, _2 : pred != 0 if label == 1 else pred == label
test = MFT(t.data, Expect.single(allow_for_neutral), labels=t.labels, templates=t.templates)
suite.add(test, 'Q & A: no', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

In [69]:
t = editor.template('Do I think {it} {air_noun} {be} {neutral_adj}? No', it=['that', 'this', 'the'], be=['is', 'was'], save=True)
t += editor.template('Do I think {it} {be} {a:neutral_adj} {air_noun}? No', it=['it', 'this', 'that'], be=['is', 'was'], save=True)
t += editor.template('Did {i} {neutral_verb_present} {the} {air_noun}? No', i=['I', 'we'], the=['this', 'that', 'the'], save=True)
test = MFT(t.data, labels=1, templates=t.templates)
suite.add(test, 'Q & A: no (neutral)', 'SRL', 'TODO_DESCRIPTION')
# test = MFT(data, labels=2)
# test.run(new_pp)
# test.summary(n=3)

In [6]:
suite.to_raw_file('/tmp/temp2', n=500, seed=1)

In [103]:
for test in suite.tests:
    suite.tests[test].name = test
    suite.tests[test].description = suite.info[test]['description]']
    suite.tests[test].capability = suite.info[test]['capability']

In [12]:
suite.save('/home/marcotcr/tmp/sentiment_suite.pkl')

In [5]:
# suite = TestSuite.from_file('/home/marcotcr/tmp/sentiment_suite.pkl')
suite = TestSuite.from_file('/home/marcotcr/work/checklist/release_data/sentiment/sentiment_suite.pkl')

In [11]:
texts = open('/tmp/temp2', 'r').read().splitlines()
preds, confs = new_pp(texts)
open('/tmp/bert', 'w').write('\n'.join(['%d %f %f %f' % (pred, *c) for pred, c in zip(preds, confs)]))

2536629

In [21]:

texts = open('/tmp/temp2', 'r').read().splitlines()
preds, confs = new_pp(texts)
open('/tmp/roberta', 'w').write('\n'.join(['%d %f %f %f' % (pred, *c) for pred, c in zip(preds, confs)]))

2536629

In [121]:
# t = 'simple negations: I thought x was positive, but it was not (should be negative)'
# print('\n'.join(np.random.choice([str(suite.tests[t].data[i]) for i in suite.tests[t].run_idxs], 10)))

In [22]:
# suite.run_from_file('/tmp/b', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [74]:
# suite.run_from_file('/tmp/azure', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [75]:
# suite.run_from_file('/tmp/google', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [76]:

# suite.run_from_file('/tmp/amazon', file_format='pred_and_softmax', overwrite=True)
# suite.summary(n=3)

In [77]:
# suite.run(new_pp, n=300, overwrite=True)

In [78]:
# suite.summary(n=3)